In [16]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

In [17]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL = pd.concat([df_SQL, df_SQL_int], ignore_index= True)

    df_SQL = df_SQL.groupby("Fecha").sum()

    return df_SQL

In [46]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

#El mejor modelo es 10-Institucional-A
filtro_distrito = 10
filtro_tipozona = "Institucional"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                    (df_clusters["Codigo distrito"] == filtro_distrito) &
                    (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                    (df_clusters["Letra cluster"] == filtro_cluster)
                ]

In [36]:
df_clusters_EDA = df_clusters.groupby(["Codigo distrito", "Nombre tipo zona", "Letra cluster"]).agg({"Nit cliente-sucursal": np.size}).reset_index()

In [38]:
df_clusters_EDA.head(20)

,Codigo distrito,Nombre tipo zona,Letra cluster,Nit cliente-sucursal
0,10,Artes graficas,A,148
1,10,Artes graficas,B,129
2,10,Artes graficas,C,90
3,10,Artes graficas,D,77
4,10,Artes graficas,E,79
5,10,Institucional,A,112
6,10,Institucional,B,213
7,10,Institucional,C,251
8,10,Institucional,D,283
9,10,Institucional,E,218


In [32]:
print(df_clusters["Codigo distrito"].unique())
print(df_clusters["Nombre tipo zona"].unique())

array([10, 15, 17, 20, 25, 40, 45, 50, 52, 53, 55, 60, 65, 70, 71, 75, 90],
      dtype=int64)

In [47]:
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [48]:
ventas_cluster = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [49]:
s = setup(ventas_cluster, fh = 6, session_id = 42)

In [59]:
best.get_params()

{'additive_only': False,
 'allow_multiplicative_trend': False,
 'auto': False,
 'bounds': None,
 'callback': None,
 'damped_trend': False,
 'dates': None,
 'disp': False,
 'error': 'add',
 'freq': None,
 'full_output': True,
 'ignore_inf_ic': True,
 'information_criterion': 'aic',
 'initial_level': None,
 'initial_seasonal': None,
 'initial_trend': None,
 'initialization_method': 'estimated',
 'maxiter': 1000,
 'missing': 'none',
 'n_jobs': None,
 'random_state': None,
 'restrict': True,
 'return_params': False,
 'seasonal': 'mul',
 'sp': 4,
 'start_params': None,
 'trend': 'add'}

In [50]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:43:10
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [51]:
plot_model(best, plot = 'forecast')

In [60]:
plot_model(best, plot = 'diagnostics')

In [61]:
plot_model(best, plot = 'insample')

In [62]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh': 9})

### Prediction

In [63]:
# predict on test set
holdout_pred = predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,ETS,0.8011,0.6958,212382276.7247,243524062.1478,0.0898,0.0897,0.3629


In [64]:
# generate forecast for 36 period in future
predict_model(best, fh = 9)

,y_pred
2022-10,2.249566e+09
2022-11,2.485196e+09
2022-12,2.543633e+09
2023-01,2.183429e+09
2023-02,2.330597e+09
2023-03,2.573915e+09
2023-04,2.633636e+09
2023-05,2.260009e+09
2023-06,2.411627e+09
